# Calcul du tenseur d'Eshelby dans le cas des ellipses

## I Cas Isotropes

### 1) Cas sphéroïdaux

Définition du tenseur d'Eshelby dans le repère de l'Ellipse

In [3]:
from classes import *
import numpy as np

def g_prolate(ar) : 
    X = ar*np.sqrt(ar**2-1)
    Y = np.power(ar**2-1,3/2)
    return ar * X/Y - ar*np.cos(ar)

def g_oblate(ar) : 
    X = -ar*np.sqrt(1-ar**2)
    Y = np.power(1-ar**2,3/2)
    return ar * X/Y - ar*np.cos(ar)

def Eshelby_tensor_sph(microstructure) : 
    
     ## Détermination du type de spheroïde
    dict_inclusions = microstructure.dict_inclusions
    inclusion = list(dict_inclusions.keys())[0] #Inclusion unique ici
    ar = inclusion.aspect_ratio
    Cm = microstructure.matrix_behavior
    nu = Cm['nu']
    S = np.zeros((6,6))
    
    if  inclusion.aspect_ratio == 1 : ## l'inclusion est une sphère
        a = (7-5*nu) / (15 *(1-nu))
        b =  2*(4-5*nu) / (15 *(1-nu))
        c =  (5*nu-1) / (15 *(1-nu))
        
         ## Calcul du tenseur d'Eshelby pour une inclusion spherique
        S[0,0] = a
        S[1,1] = a
        S[2,2] = a
        S[3,3] = b
        S[4,4] = b
        S[5,5] = b     
        S[0,1] = c
        S[1,0] = c
        S[2,0] = c
        S[0,2] = c
        S[1,2] = c
        
        return S 
    
    elif ar < 1 :
        g = g_oblate(inclusion.aspect_ratio)
    else : 
        g = g_prolate(inclusion.aspect_ratio)
        
    ## Calcul du tenseur d'Eshelby
     
    S[0,0] = 1/(2*(1-nu)) * (1-2*nu + (3*ar**2-1)/(ar**2-1) - g*(1-2*nu + 3*ar**2/(ar**2-1)))
    S[0,1] = -1/(2*(1-nu)) * (1-2*nu + 1/(ar**2-1) + g*(1-2*nu + 3/(ar**2-1)))
    S[1,0] = -ar**2/(2*(1-nu))*(ar**2-1) + g/(4*(1-nu))*(3*ar**2/(ar**2-1) - (1-2*nu))
    S[2,0] = S[1,0]
    S[1,1] = 3*ar**2/(8*(1-nu))*(ar**2-1) + g/(4*(1-nu))*(1-2*nu - 9/(4*(ar**2-1)))
    S[0,2] = S[0,1]
    S[1,2] = 1/(4*(1-nu)) * ( ar**2/(2*(ar**2-1)) - g* (1-2*nu + 3/(4*(ar**2-1))))
    S[2,2] = S[1,1]
    S[3,3] = 2/(4*(1-nu)) * (1-2*nu - (ar**2+1)/(ar**2-1) - g/2 * (1-2*nu + (3*ar**2+1)/(ar**2-1)))
    S[4,4] = S[3,3]
    S[5,5] = 2/(4*(1-nu)) * (ar**2/(2*(ar**2-1)) + g * (1-2*nu - 3/(4*(ar**2-1))))
    
    return S
    
    
    
    

### Tests 

In [4]:
## OBLATE
f_inclusion = 0.1
Km,Gm = 1,2
Kf,Gf = 10,20
inclusion_behavior = {"K":Kf, "G":Gf}        
matrix_behavior = {"K":Km, "G":Gm} 
inclusion = Inclusion(1 ,inclusion_behavior,0.2)
microstructure = Microstructure(matrix_behavior,{inclusion : f_inclusion})
print(Eshelby_tensor(microstructure))

## PROLATE
f_inclusion = 0.1
inclusion_behavior = {"K":Kf, "G":Gf}        
matrix_behavior = {"K":Km, "G":Gm} 
inclusion = Inclusion(2 ,inclusion_behavior,1.8)
microstructure = Microstructure(matrix_behavior,{inclusion : f_inclusion})
print(Eshelby_tensor(microstructure))

[[ 1.07826029 -0.27993968 -0.27993968  0.          0.          0.        ]
 [ 0.08902863 -0.20451783  0.01788526  0.          0.          0.        ]
 [ 0.08902863  0.         -0.20451783  0.          0.          0.        ]
 [ 0.          0.          0.          1.03967939  0.          0.        ]
 [ 0.          0.          0.          0.          1.03967939  0.        ]
 [ 0.          0.          0.          0.          0.         -0.22351673]]
[[-2.3566778  -2.88990902 -2.88990902  0.          0.          0.        ]
 [-1.97513485  2.55663533 -0.48283674  0.          0.          0.        ]
 [-1.97513485  0.          2.55663533  0.          0.          0.        ]
 [ 0.          0.          0.         -2.83899153  0.          0.        ]
 [ 0.          0.          0.          0.         -2.83899153  0.        ]
 [ 0.          0.          0.          0.          0.          1.0583909 ]]


### 2) Cas Isotrope Général

Référence : [Toshio Mura, 1987 ,  Micromechanics of defects in solids ]

Par. 7.3.6, p.91




In [45]:
from classes import *
import numpy as np
from scipy.integrate import quad
from numpy import pi,sqrt

def C3333_to_66 (G) : 
    "Passe d'un tenseur  de comportement (C) 3x3x3x3 a un tenseur de comportement 6x6"
    F=np.zeros((6,6))
    for i in range(3):
        for j in range(3):
            F[i,j] = G[i,i,j,j]
            
        F[i,5]=(G[i,i,0,1]+G[i,i,1,0])/2.
        F[i,3]=(G[i,i,1,2]+G[i,i,2,1])/2. 
        F[i,4]=(G[i,i,2,0]+G[i,i,0,2])/2. 
        F[3,i]=(G[1,2,i,i]+G[2,1,i,i])/2. 
        F[4,i]=(G[0,2,i,i]+G[2,0,i,i])/2.
        F[5,i]=(G[0,1,i,i]+G[1,0,i,i])/2.

    F[4,4]=(G[0,2,0,2]+G[2,0,0,2]+G[0,2,2,0]+G[2,0,2,0])/4. 
    F[3,3]=(G[1,2,1,2]+G[2,1,1,2]+G[1,2,2,1]+G[2,1,2,1])/4.  
    F[5,5]=(G[0,1,0,1]+G[1,0,0,1]+G[0,1,1,0]+G[1,0,1,0])/4.  
    F[4,3]=(G[0,2,1,2]+G[2,0,1,2]+G[0,2,2,1]+G[2,0,2,1])/4.  
    F[4,5]=(G[0,2,1,0]+G[2,0,1,0]+G[0,2,0,1]+G[2,0,0,1])/4.  
    F[3,4]=(G[1,2,0,2]+G[2,1,0,2]+G[1,2,2,0]+G[2,1,2,0])/4.  
    F[5,4]=(G[0,1,0,2]+G[1,0,0,2]+G[0,1,2,0]+G[1,0,2,0])/4.  
    F[3,5]=(G[1,2,1,0]+G[2,1,1,0]+G[1,2,0,1]+G[2,1,0,1])/4.   
    F[5,3]=(G[0,1,1,2]+G[1,0,1,2]+G[0,1,2,1]+G[1,0,2,1])/4. 
    
    return F

def check_hypothesis(A) : 
    a1,a2,a3 = A
    return not (a1==a2 or a1==a3 or a2==a3)

def non_zeros(S,i,j,k,l) : 
    if S[i,j,k,l] != 0 : 
        return S[i,j,k,l]
    if S[i,j,l,k] != 0 : 
        return S[i,j,l,k]
    if S[j,i,k,l] != 0 : 
        return S[j,i,k,l]
    if S[j,i,l,k] != 0 : 
        return S[j,i,l,k]
    return 0

def Symetry(S) :
    n = S.shape[0]
    for i in range (n) :
        for j in range (n) : 
            for k in range(n) :
                for l in range(n) :
                    val = non_zeros(S,i,j,k,l)
                    S[i,j,k,l] = val
                    S[i,j,l,k] = val
                    S[j,i,k,l] = val
                    S[j,i,l,k] = val
    return S

def Cyclic_permutation(S) : 
    n = S.shape[0]
    for i in range (n) :
        for j in range (n) : 
            for k in range(n) :
                for l in range(n) :     
                    i1,j1,k1,l1 = (i+1)%3,(j+1)%3,(k+1)%3,(l+1)%3
                    i2,j2,k2,l2 = (i+2)%3,(j+2)%3,(k+2)%3,(l+2)%3
                    S[i1,j1,k1,l1] = S[i,j,k,l]
                    S[i2,j2,k2,l2] = S[i,j,k,l]
    return S

def delta(A,s) :
    a1,a2,a3 = A
    return sqrt((a1**2+s)*(a2**2+s)*(a3**2+s))
    
def Eshelby_tensor_ell(A,Cm) :
    
    # Recupération des paramètres geometriques de l'ellipse 
    a1,a2,a3 = A
    nu = Cm['nu']
    compatible = check_hypothesis(A)
    if not compatible : 
        raise NameError("Unable to treat spheroidal cases")

    # Calcul du tenseur d'Eshelby formulation sans relation d'ordre entre a1,a2,a3
    # Il y a une autre formulation avec a1>a2>a3 qui évite les intégrales généralisées (interessant pour les complexes)
    
    I1 = 2*pi*a1*a2*a3*quad(lambda x : 1/((a1**2+x) * delta(A,x)) , 0 , np.inf)[0]
    I2 = 2*pi*a1*a2*a3*quad(lambda x : 1/((a2**2+x) * delta(A,x)) , 0 , np.inf)[0]
    
    I11 = 2*pi*a1*a2*a3*quad(lambda x : 1/((a1**2+x)**2 * delta(A,x)) , 0 , np.inf)[0]  
    I12 = 2*pi*a1*a2*a3*quad(lambda x : 1/((a1**2+x)*(a2**2+x) * delta(A,x)) , 0 , np.inf)[0]
    I13 = 2*pi*a1*a2*a3*quad(lambda x : 1/((a3**2+x)*(a1**2+x) * delta(A,x)) , 0 , np.inf)[0]
    
    #I3 = 2*pi*a1*a2*a3*quad(lambda x : 1/((a3**2+x) * delta(A,x)) , 0 , np.sup)
    #I22 = 2*pi*a1*a2*a3*quad(lambda x : 1/((a2**2+x)**2 * delta(A,x)) , 0 , np.sup)
    #I33 = 2*pi*a1*a2*a3*quad(lambda x : 1/((a3**2+x)**2 * delta(A,x)) , 0 , np.sup)
    #I23 = 2*pi*a1*a2*a3*quad(lambda x : 1/((a2**2+x)*(a3**2+x) * delta(A,x)) , 0 , np.sup)
 
    S = np.zeros((3,3,3,3))
    S[0,0,0,0] = 3*a1**2*I11 / (8*pi*(1-nu)) + I1 * (1-2*nu)/(8*pi*(1-nu))
    S[0,0,1,1] = a2**2*I12/(8*pi*(1-nu)) -  I1 * (1-2*nu)/(8*pi*(1-nu))
    S[0,0,2,2] = a3**2*I13/(8*pi*(1-nu)) -  I1 * (1-2*nu)/(8*pi*(1-nu))
    S[0,1,0,1] = (a1**2+a2**2)*I12/(16*pi*(1-nu)) + (1-2*nu)/(16*pi*(1-nu))*(I1+I2)
    
    S = Cyclic_permutation(S)
    S = Symetry(S)
    
    return S
    
    
    
    
    


### TEST

In [46]:
Cm = {'nu':0.2}
A = 3,2,1
S = Eshelby_tensor_ell(A,Cm)
C1 = C3333_to_66(S)
C2 = C3333_to_66(Symetry(S))
print(S)
print(C1)
print(C2)

[[[[ 0.2634183   0.          0.        ]
   [ 0.         -0.00318609  0.        ]
   [ 0.          0.         -0.02578116]]

  [[ 0.          0.1694661   0.        ]
   [ 0.1694661   0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.1694661 ]
   [ 0.          0.          0.        ]
   [ 0.1694661   0.          0.        ]]]


 [[[ 0.          0.1694661   0.        ]
   [ 0.1694661   0.          0.        ]
   [ 0.          0.          0.        ]]

  [[-0.02578116  0.          0.        ]
   [ 0.          0.2634183   0.        ]
   [ 0.          0.         -0.00318609]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.1694661 ]
   [ 0.          0.1694661   0.        ]]]


 [[[ 0.          0.          0.1694661 ]
   [ 0.          0.          0.        ]
   [ 0.1694661   0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.1694661 ]
   [ 0.          0.1694661   0.        